In [ ]:
import numpy as np

In [ ]:
onehot_vector = np.identity(3)
print(onehot_vector) #Identity
print(onehot_vector[[0,1,0,2]]) # Rows 0,1,0,2

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


La CMM è una struttura dati che permette di associare coppie di input-output usando una matrice di correlazione come meccanismo di memoria. Si tratta di una matrice che registra, in forma di pesi, le relazioni tra input (ad esempio, una sequenza di parole) e output attesi (ad esempio, la parola successiva nella sequenza). Questo è particolarmente utile per applicazioni in cui bisogna "ricordare" delle transizioni o associazioni.

Come funziona la CMM?
Creazione dei Vettori One-Hot: I simboli (parole) sono rappresentati come vettori one-hot, in cui ciascun vettore ha tutti valori nulli tranne un 1 in corrispondenza della parola specifica.

Costruzione della CMM: La matrice CMM viene costruita tramite il prodotto esterno di ciascuna sequenza di input (trasformata in un vettore concatenato) con il vettore one-hot che rappresenta la parola successiva attesa.

In [ ]:
symbols = ["a","b","c"]
onehot_vectors = np.identity(len(symbols))
len_seq = 3

### a b a --> b
### a a b --> a
### a c b --> c
### c a b --> a

def i(symbol):
    return symbols.index(symbol)

def seq(s1,s2,s3):
    return onehot_vectors[[i(s1),i(s2),i(s3)]].reshape(1,len(symbols)*len_seq) #Transform the vector in One-Hot vector, from 3x3 to 1x9

#print(seq("a","b","a"))

CMM = np.matmul(np.transpose(seq("a","b","a")),onehot_vectors[[i("b")]]) +\
    np.matmul(np.transpose(seq("a","a","b")),onehot_vectors[[i("a")]]) +\
    np.matmul(np.transpose(seq("a","c","b")),onehot_vectors[[i("c")]]) +\
    np.matmul(np.transpose(seq("c","a","b")),onehot_vectors[[i("a")]])

#print(CMM)

(s1,s2,s3) = ("a","b","a")
out = np.matmul(seq(s1,s2,s3),CMM)

print(out)
print("Sequence" + str((s1,s2,s3)) + " --> predicted symbol --> " +  symbols[np.argmax(out)])

[[1. 3. 1.]]
Sequence('a', 'b', 'a') --> predicted symbol --> b


In [6]:
from numpy.random import normal

symbols = ["a","b","c","d","e","f","g"]
pairs = [("a","a"),("b","c"),("d","f"),("e","f"),("f","d")] #Defining key-value associations

In [7]:
d = 100 #Dimension
dist_vecs = normal(0, 1.0/np.sqrt(d), (len(symbols), d)) #Matrix 7x100 (7 symbols)
CMM = np.zeros((d, d))

In [8]:
for (k,v) in pairs:
  #Dist_vecs[i(k)] ci da un vettore che descrive k in dim: (1,d)

    CMM = CMM + np.matmul(np.transpose([dist_vecs[i(k)]]), [dist_vecs[i(v)]])
  #Scalar product between k and w, so we are injecting information in CMM

In [12]:
succ = 0.0
for (k,v) in pairs:
    pred_v = np.matmul([dist_vecs[i(k)]], CMM) #(1,d) * (d,d) = (1,d)
    #Questo ci dice come il simbolo k si relaziona con tutte le
    #Associazioni memorizzate. Ora però se rimoltiplico per i vettori, ottengo
    #Un punteggio per ogni vettore.
    #(1,d) * (len(symbols),d)^T = (1, len(symbols)) Maxarg, è il migliore.
    pred_v = symbols[np.argmax(np.matmul(np.matmul([dist_vecs[i(k)]], CMM), np.transpose(dist_vecs)))]
    if pred_v == v:
        succ += 1.0
print(succ / len(pairs))

1.0
